In [6]:
import clip
import torch
from PIL import Image
import spacy
import pandas as  pd

In [17]:
# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")


In [18]:
# Load the CLIP model onto the CPU
model, preprocess = clip.load("ViT-B/32", device='cpu')

# Load the NLP model (spaCy)
nlp = spacy.load("en_core_web_sm")

# Setting up a base directory for the images 
BASE_IMG_DIR = "Example Data-20240208T214429Z-001\Example Data\exported"
image_name = "IMG_3155.JPG"
# Prepare the image
image_path = f"{BASE_IMG_DIR}/{image_name}"

image = preprocess(Image.open(image_path)).unsqueeze(0).to('cpu')

# Accept user input for text prompt and smart story description
user_text_prompt = "a photo of people standing outside"
smart_story_description = "Three individuals outdoors, smiling at the camera and pointing at a park bench."

# Process the texts with spaCy for named entity recognition (NER)
user_doc = nlp(user_text_prompt)
story_doc = nlp(smart_story_description)

# Extract entities, or use the original text if no entities are found
user_entities = ' '.join([ent.text for ent in user_doc.ents]) if user_doc.ents else user_text_prompt
story_entities = ' '.join([ent.text for ent in story_doc.ents]) if story_doc.ents else smart_story_description

# Tokenize the refined texts
text_inputs = clip.tokenize([user_entities, story_entities]).to('cpu')

# Calculate the features with CLIP
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_inputs)

    # Normalize the features to unit vectors
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

# Compute cosine similarity
cosine_similarities = torch.matmul(image_features, text_features.T).cpu().numpy().flatten()

# Print the cosine similarity scores
print("Cosine  Similarity Score -1 to 1")
print("1 indicates identical directionality (very similar),")
print("0 indicates orthogonality (not similar),")
print("and -1 indicates opposite directionality (very dissimilar)."  )
print(f"Cosine similarity score for user text prompt: {cosine_similarities[0]}")
print(f"Cosine similarity score for smart story description: {cosine_similarities[1]}")

<>:8: SyntaxWarning: invalid escape sequence '\E'
<>:8: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Hussein\AppData\Local\Temp\ipykernel_13552\1133486942.py:8: SyntaxWarning: invalid escape sequence '\E'
  BASE_IMG_DIR = "Example Data-20240208T214429Z-001\Example Data\exported"


Cosine  Similarity Score -1 to 1
1 indicates identical directionality (very similar),
0 indicates orthogonality (not similar),
and -1 indicates opposite directionality (very dissimilar).
Cosine similarity score for user text prompt: 0.2476407289505005
Cosine similarity score for smart story description: 0.18213330209255219


In [19]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import numpy as np

class LoadCocoDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform

        # List all image files in the directory
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        
        # Load label
        label_path = os.path.join(self.label_dir, self.image_files[idx].replace('.jpg', '.txt').replace('.png', '.txt'))
        
        # Assuming the labels are space-separated values: class x_center y_center width height
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                for line in file:
                    labels.append(np.fromstring(line, sep=' '))

        labels = np.array(labels)  # Convert to a NumPy array

        if self.transform:
            image = self.transform(image)
        
        return image, labels

# Define your transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Initialize your dataset
coco_dataset = LoadCocoDataset(
    image_dir='datasets/coco128/images',
    label_dir='datasets/coco128/labels/train2017',
    transform=transform
)

# Initialize DataLoader
data_loader = DataLoader(coco_dataset, batch_size=4, shuffle=True)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [20]:
for images, labels in data_loader:
    print(images, labels)# - images: a batch of images
    # - labels: the corresponding labels for each image
    


NameError: name 'data_loader' is not defined

In [2]:
import pandas as pd
import os
import pytesseract
from PIL import Image
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
df = pd.read_excel('Example Data-20240208T214429Z-001/Example Data/ImageLabels.xlsx', usecols=['Image Name', 'Confidence', 'Instance Count', 'Label'])



# Append the instance counts >=1 to the label, e.g: '1 Human'

# Filter based on the criteria
filtered_df = df[(df['Confidence'] >= 70)]

# Append the instance counts to the label, e.g: '1 Human'
filtered_df['Label'] = filtered_df['Instance Count'].astype(str) + ' ' + filtered_df['Label']

# Aggregate labels for each image
aggregated_labels = filtered_df.groupby('Image Name')['Label'].apply(lambda x: ', '.join(x)).reset_index()

# Display the aggregated dataframe
aggregated_labels.head()

C:\Users\Hussein\AppData\Local\Temp\ipykernel_14576\2903431629.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Label'] = filtered_df['Instance Count'].astype(str) + ' ' + filtered_df['Label']


,Image Name,Label
0,(1) 18380579401063495.JPG,1 Motor Bike
1,(1) @GreyCupFestival - 109th Grey Cup.jpeg,"2 Adult, 1 Female, 7 Person, 1 Woman, 1 Male, ..."
2,(10) 17887803224903630.jpeg,"0 Animal, 1 Horse, 0 Horseback Riding, 0 Leisu..."
3,(12) 17985809330117499.jpeg,"0 Fun, 0 Vacation, 1 Person, 0 Road Trip, 1 He..."
4,(13) 18013990822817757.jpeg,"0 Clothing, 0 Coat, 0 Jacket, 0 Vest, 0 Shirt,..."


In [44]:
# Converting to a dictionary for ease of access
image_labels_dict = aggregated_labels.set_index('Image Name')['Label'].to_dict()

# Displaying the dictionary
image_labels_dict

{'(1) 18380579401063495.JPG': '1 Motor Bike',
 '(1) @GreyCupFestival - 109th Grey Cup.jpeg': '2 Adult, 1 Female, 7 Person, 1 Woman, 1 Male, 1 Man, 1 Helmet, 0 Crowd, 0 Clothing, 1 Coat, 0 Footwear, 1 Shoe, 0 Face, 0 Head, 0 Accessories, 0 Bag, 1 Handbag, 1 Glasses',
 '(10) 17887803224903630.jpeg': '0 Animal, 1 Horse, 0 Horseback Riding, 0 Leisure Activities, 0 Mammal, 1 Person, 1 Adult, 1 Female, 1 Woman',
 '(12) 17985809330117499.jpeg': '0 Fun, 0 Vacation, 1 Person, 0 Road Trip, 1 Helmet, 0 Nature, 0 Outdoors, 0 Sky, 1 Motorcycle, 0 Transportation, 0 Vehicle',
 '(13) 18013990822817757.jpeg': '0 Clothing, 0 Coat, 0 Jacket, 0 Vest, 0 Shirt, 0 T-Shirt, 0 Document, 0 Receipt, 0 Text, 0 Id Cards, 3 Passport, 0 Symbol, 0 Face, 0 Head, 1 Person, 1 Credit Card, 1 Business Card, 0 Paper',
 '(14) 17993584322154200.jpeg': '0 Advertisement, 2 Person, 0 Photography, 0 Animal, 6 Bird, 0 Nature, 0 Outdoors, 0 Poster, 0 Clothing, 1 Glove, 0 Wildlife, 0 Footwear, 2 Shoe, 1 Adult, 1 Bride, 1 Female, 0 

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

# Example user inputs
user_text_prompt = "I went snow boarding the mountains"
smart_story_description = "Rockey Mountains"

# Step 4: Tokenize the user_text_prompt, smart_story_descriptions, and the image labels.

# Combine the user inputs and the image labels into a single list for tokenization
texts_to_tokenize = [user_text_prompt, smart_story_description] + list(image_labels_dict.values())

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Tokenize the texts
tokenized_texts = vectorizer.fit_transform(texts_to_tokenize)

# Convert the tokenized texts to an array for easier viewing and further processing
tokenized_texts_array = tokenized_texts.toarray()

# Show the shape of the array to confirm the tokenization
tokenized_texts_array.shape, vectorizer.get_feature_names_out()[:10]

((33, 169),
 array(['10', '11', '12', '13', '14', 'accessories', 'activities', 'adult',
        'adventure', 'advertisement'], dtype=object))

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

# and that the first two rows correspond to user_text_prompt and smart_story_description respectively

# Extract vectors for user inputs
user_text_vector = tokenized_texts_array[0].reshape(1, -1)  # User text prompt
story_desc_vector = tokenized_texts_array[1].reshape(1, -1)  # Smart story description

# Calculate cosine similarity between user inputs and all image labels
cos_sim_user_text = cosine_similarity(user_text_vector, tokenized_texts_array[2:])  # Skipping the first two
cos_sim_story_desc = cosine_similarity(story_desc_vector, tokenized_texts_array[2:])

# The result is a list of similarity scores between the user inputs and each image label vector
# Let's assume we want to find the image labels most similar to the user_text_prompt
most_similar_index = cos_sim_user_text.argmax()  # This gets the index of the highest similarity score
most_similar_image = list(image_labels_dict.keys())[most_similar_index]  # Retrieve the corresponding image name

most_similar_image, cos_sim_user_text[0, most_similar_index]

('(2) 18040499875507660.jpeg', 0.11547005383792514)

**Fast Text**

In [58]:
! pip install fasttext
import fasttext
import fasttext.util

# Load pre-trained FastText model
# fasttext.util.download_model('en', if_exists='ignore')  # Uncomment if you need to download the model
ft_model = fasttext.load_model('cc.en.300.bin')

def text_to_embedding(text, model):
    """
    Converts a text to an embedding by averaging the embeddings of the words in the text.
    """
    words = text.split()
    embeddings = [model.get_word_vector(word) for word in words]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.get_dimension())

# Assuming aggregated_labels is your DataFrame from previous steps
# Convert each label into an embedding
aggregated_labels['Embedding'] = aggregated_labels['Label'].apply(lambda x: text_to_embedding(x, ft_model))

# Example user input
user_text_prompt = "This my cat is he is the best, cats are better than dogs"
user_embedding = text_to_embedding(user_text_prompt, ft_model)

# Calculate cosine similarity between user input and image labels
def cosine_similarity(a, b):
    """
    Compute the cosine similarity between two vectors.
    """
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Adding a similarity score for each image label
aggregated_labels['Similarity'] = aggregated_labels['Embedding'].apply(lambda x: cosine_similarity(x, user_embedding))

# Find the most similar images
most_similar_images = aggregated_labels.sort_values(by='Similarity', ascending=False).head()

# Display the most similar images
most_similar_images[['Image Name', 'Label', 'Similarity']]

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for fasttext
Failed to build fasttext


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [43 lines of output]
      C:\Python312\Lib\site-packages\setuptools\dist.py:472: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2024-Sep-26, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        opt = self.warn_dash_deprecation(opt, section)
      running bdist_wh

ModuleNotFoundError: No module named 'fasttext'